# Tutorial "Text Mining"
Le projet consiste à analyse un site d’offre d’emploi de Marocain, en extraire les demandes et les offres d’emploi et créer des profils de compétence selon le CV ou la description du poste.

Dans Cette partie du projet on va analyse les offres d'emploi.

## Le plan 
### Etape 1: Load the data & Import the library 
   Apres l'extraction des données dans un site d’offre d’emploi sous la forme de csv. 

### Etape 2 : Data cleaning
1 - Word_count
       
2 - Common word removal
         
    2 - 1 Calculer les frequences des mots.
           
    2 - 2 suprimer les mots commun.

### Etape 3 : Analyse de données            

    1- Utiliser TF*IDF afin de pondérer les mots de chaque offre/demande collecté. Cette étape consiste à indexer les annonces en créant un vecteur (mot, poids) pour chaque annonce.
    
    2- Créer un index global dans lequel vous mettez les compétences les plus demandé (ou offertes) à la limite de 20 compétence.
    
    3 - Créer une page web qui montre un tableau de bord résumant l’état du marché de l’emploi (avec des statistiques comme :
        
        a. Les compétences les plus demandées selon la ville
        
        b. Nombre des competences demander selon les ville

### Etape 1: Load the data & Import the library 

In [1]:
#import io
import  pandas as pd # Pour Data Cleaning 
import numpy as np # Pour la competition matricial 
from collections import defaultdict # On a besoin lors de traitement 


In [2]:
# Etape 1: charge les data dans une  DataFrame et affiche le header(les premiers 6 lines) 

data1 = pd.read_csv('off.csv', sep=";",encoding='ISO-8859-1',error_bad_lines=False)
data1.head()

,offre,date,company,description,ville,competences_souhaites
0,Ingénieur-de Développement,12.03.2019,HPS,Nous recrutons un Ingénieur-de DéveloppementMi,Casablanca,JAVA/JEE
1,Développeur Informatique Angular / Java confirmé,12.03.2019,BEREXIA,BEREXIA Conseil société d'ingénierie informatique,Casablanca,Angular React Web
2,DÉVELOPPEUR INFORMATIQUE JAVA JEE / ANGULAR,12.03.2019,NEXTON CONSULTING,Nous recherchons un Développeur Informatique,Rabat,Java JEE
3,INGÉNIEUR-INFORMATIQUE DÉVELOPPEMENT,12.03.2019,KROSSONE,Pour le compte d'une SII sur Casablanca nous r...,Casablanca,GestionDeProjet Java
4,Graphiste (H/F),12.03.2019,FINAPPS,Pour le compte d'une SII sur Casablanca nous r...,Rabat,infographie3D


### Etape 2 : Data cleaning

1 - Word_count    

In [3]:
data1['word_count'] = data1['competences_souhaites'].apply(lambda x: len(str(x).split(",")))
data1['competences_souhaites'] = data1['competences_souhaites'] + ' ' +  data1['offre']
data1[['competences_souhaites','word_count']].head()


,competences_souhaites,word_count
0,JAVA/JEE Ingénieur-de Développement,1
1,Angular React Web Développeur Informatique An...,1
2,Java JEE DÉVELOPPEUR INFORMATIQUE JAVA JEE / ...,1
3,GestionDeProjet Java INGÉNIEUR-INFORMATIQUE DÉ...,1
4,infographie3D Graphiste (H/F),1


2 - Common word removal
         
    2 - 1 Calculer les frequences des mots.
           
    

In [4]:
freq = pd.Series(' '.join(data1['competences_souhaites']).split()).value_counts()[:27]
freq

Java            18
de               7
/                7
JEE              7
JAVA/JEE         6
Développeur      6
BI               6
(H/F)            5
Angular          5
Informatique     5
C#               4
Consultant       4
Support          4
Team             4
Data             4
PLSQL            4
C                4
JAVA             4
Mobile           4
Télécoms         3
Oracle           3
Architecte       3
et               3
Android          3
comptabilité     3
Gestion          3
INGÉNIEUR        3
dtype: int64

2 - 2 suprimer les mots commun.

In [5]:
import nltk

from textblob import TextBlob

from nltk.corpus import stopwords
stop = stopwords.words('french')
data1['competences_souhaites'] = data1['competences_souhaites'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data1['competences_souhaites'] = data1['competences_souhaites'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

data1['competences_souhaites'].head(20)


0                   java/jee ingénieur-de développement
1     angular react web développeur informatique ang...
2     java jee développeur informatique java jee / a...
3     gestiondeprojet java ingénieur-informatique dé...
4                         infographie3d graphiste (h/f)
5                    android ios tl mobile(ios/android)
6              android ios ingénieur-mobile ios/android
7     gestiondeprojet mobile team leader mobile ios/...
8                         java jee team leader java/jee
9                     java jee java architecte java/jee
10                              bi python consultant bi
11                    bigdata hbase consultant big data
12    commerceinternationale marketing commercial se...
13    angular java ingénieur-informatique java / ang...
14                              cp chef projet pmo / it
15    c# .net développeur informatique confirmé c# .net
16      c# .net ingénieur-informatique confirmé c# .net
17              java web responsable informatiqu

### Etape 3 : Analyse de données            

    1- Utiliser TF*IDF afin de pondérer les mots de chaque offre/demande collecté. Cette étape consiste à indexer les annonces en créant un vecteur (mot, poids) pour chaque annonce.
    
   

In [6]:
dataRabat = data1[data1['ville']=='Rabat']
dataCasa = data1[data1['ville']=='Casablanca']
dataCasa.head()
dataRabat.head()

,offre,date,company,description,ville,competences_souhaites,word_count
2,DÉVELOPPEUR INFORMATIQUE JAVA JEE / ANGULAR,12.03.2019,NEXTON CONSULTING,Nous recherchons un Développeur Informatique,Rabat,java jee développeur informatique java jee / a...,1
4,Graphiste (H/F),12.03.2019,FINAPPS,Pour le compte d'une SII sur Casablanca nous r...,Rabat,infographie3d graphiste (h/f),1
14,Chef de Projet PMO / IT,11.03.2019,VISION BUSINESS CONSULTING,Au sein de la DSI de lun de nos clients,Rabat,cp chef projet pmo / it,1
15,Développeur Informatique Confirmé C# .NET,11.03.2019,ONE TECHNOLOGIE,One Technologie société spécialisée dans les a...,Rabat,c# .net développeur informatique confirmé c# .net,1
16,Ingénieur-Informatique Confirmé C# .NET,11.03.2019,ONE TECHNOLOGIE,One Technologie société spécialisée dans linf...,Rabat,c# .net ingénieur-informatique confirmé c# .net,1


In [7]:

TextBlob(data1['competences_souhaites'][1]).words

WordList(['angular', 'react', 'web', 'développeur', 'informatique', 'angular', 'java', 'confirmé'])

Advance Text Processing

# Term frequency


In [8]:
TextBlob(data1['competences_souhaites'][0]).ngrams(1)

[WordList(['java/jee']),
 WordList(['ingénieur-de']),
 WordList(['développement'])]

TF est simplement le rapport entre le nombre de mots présents dans une phrase et la longueur de la phrase.

Par conséquent, nous pouvons généraliser la fréquence des termes comme suit:

TF = (nombre de fois où le terme T apparaît dans la ligne particulière) / (nombre de termes dans cette ligne)

Pour en savoir plus sur la fréquence des termes, consultez cet article.

Ci-dessous, j'ai essayé de vous montrer le terme tableau de fréquence d'un Data1.

In [9]:
tf1 = (data1['competences_souhaites'][1:27]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,angular,6.0
1,react,1.0
2,/,5.0
3,informatique,5.0
4,java,15.0
5,web,2.0
6,développeur,6.0
7,confirmé,5.0
8,jee,4.0
9,développement,2.0


In [10]:
for i,word in enumerate(tf1['words']):
    if (len(data1[data1['competences_souhaites'].str.contains(word)])) != 0:
        tf1.loc[i, 'idf'] = np.log(int(data1.shape[0])/(len(data1[data1['competences_souhaites'].str.contains(word)])))

print(tf1)

                     words    tf       idf
0                  angular   6.0  2.282382
1                    react   1.0  3.891820
2                        /   5.0  0.713766
3             informatique   5.0  1.589235
4                     java  15.0  1.058607
5                      web   2.0  2.505526
6              développeur   6.0  1.945910
7                 confirmé   5.0  2.100061
8                      jee   4.0  1.812379
9            développement   2.0  2.100061
10  ingénieur-informatique   3.0  2.793208
11         gestiondeprojet   2.0  3.198673
12                   (h/f)   2.0  2.282382
13               graphiste   1.0  3.891820
14           infographie3d   1.0  3.891820
15                      tl   1.0  2.282382
16                     ios   2.0  2.282382
17     mobile(ios/android)   1.0       NaN
18                 android   2.0  2.282382
19             ios/android   2.0  2.282382
20        ingénieur-mobile   1.0  3.891820
21                  mobile   2.0  2.282382
22         

/home/ou/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/ou/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']

tf1

,words,tf,idf,tfidf
0,angular,6.0,2.282382,13.694294
1,react,1.0,3.891820,3.891820
2,/,5.0,0.713766,3.568832
3,informatique,5.0,1.589235,7.946176
4,java,15.0,1.058607,15.879104
5,web,2.0,2.505526,5.011052
6,développeur,6.0,1.945910,11.675461
7,confirmé,5.0,2.100061,10.500304
8,jee,4.0,1.812379,7.249515
9,développement,2.0,2.100061,4.200122


### Pour casa et rebat TR et IDF et TFIDF

In [12]:
TFcasa = (dataCasa['competences_souhaites'][1:27]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
TFcasa.columns = ['words','tf']

TFrabat = (dataRabat['competences_souhaites'][1:27]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
TFrabat.columns = ['words','tf']

print("casa",TFcasa.head(2))
print()
print("rabat",TFrabat.head(2))


casa      words   tf
0  angular  6.0
1    react  1.0

rabat        words   tf
0      (h/f)  2.0
1  graphiste  1.0


#Inverse Document Frequency

In [13]:
# For casa

for i,word in enumerate(TFcasa['words']):
    if (len(dataCasa[dataCasa['competences_souhaites'].str.contains(word)])) != 0:
        TFcasa.loc[i, 'idf'] = np.log(int(data1.shape[0])/(len(dataCasa[dataCasa['competences_souhaites'].str.contains(word)])))

print(TFcasa)

# For Rabat
for i,word in enumerate(TFrabat['words']):
    if (len(dataRabat[dataRabat['competences_souhaites'].str.contains(word)])) != 0:
        TFrabat.loc[i, 'idf'] = np.log(int(data1.shape[0])/(len(dataRabat[dataRabat['competences_souhaites'].str.contains(word)])))

print(TFrabat)

/home/ou/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/home/ou/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/home/ou/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # This is added back by InteractiveShellApp.init_path()
/home/ou/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


                     words    tf       idf
0                  angular   6.0  2.505526
1                    react   1.0  3.891820
2                        /   4.0  0.947381
3             informatique   3.0  2.100061
4                     java  16.0  1.252763
5                      web   1.0  3.198673
6              développeur   5.0  2.505526
7                 confirmé   4.0  2.505526
8            développement   4.0  2.100061
9   ingénieur-informatique   2.0  3.198673
10         gestiondeprojet   2.0  3.198673
11                      tl   1.0  2.282382
12                     ios   2.0  2.282382
13     mobile(ios/android)   1.0       NaN
14                 android   2.0  2.282382
15             ios/android   2.0  2.282382
16        ingénieur-mobile   1.0  3.891820
17                  mobile   2.0  2.282382
18                    team   3.0  2.505526
19                  leader   3.0  2.505526
20                     jee   5.0  1.945910
21                java/jee   6.0  1.945910
22         

#Term Frequency – Inverse Document Frequency (TF-IDF)

In [14]:
#for Rabat 
TFrabat['tfidf'] = TFrabat['tf'] * TFrabat['idf']
TFrabat

#for casa 
TFcasa['tfidf'] = TFcasa['tf'] * TFcasa['idf']
TFcasa

,words,tf,idf,tfidf
0,angular,6.0,2.505526,15.033156
1,react,1.0,3.891820,3.891820
2,/,4.0,0.947381,3.789525
3,informatique,3.0,2.100061,6.300182
4,java,16.0,1.252763,20.044207
5,web,1.0,3.198673,3.198673
6,développeur,5.0,2.505526,12.527630
7,confirmé,4.0,2.505526,10.022104
8,développement,4.0,2.100061,8.400243
9,ingénieur-informatique,2.0,3.198673,6.397346


In [15]:
a=set(data1.ville)
a

{'Bouskoura', 'Casablanca', 'Rabat'}

 2- Créer un index global dans lequel vous mettez les compétences les plus demandé (ou offertes) à la limite de 20 compétence.
    

In [16]:
for x in a:
    i=0
    for x1 in data1['ville']:    
        if x==x1:
            i=i+1
    print(i,x)
            

39 Casablanca
1 Bouskoura
9 Rabat


3 - Créer une page web qui montre un tableau de bord résumant l’état du marché de l’emploi (avec des statistiques comme :
        
        a. Les compétences les plus demandées selon la ville
        
      


 #### b. Les competance les plus demander selon les villes            

In [17]:
def ComPlusDemande(TFville):
    
    a=TFville.index[TFville['tfidf']==max(TFville['tfidf'])]
                    
    return a
R,= ComPlusDemande(TFrabat)

print('la competance la plus demander ', TFrabat['words'][R],'dans la ville est Rabat la valeur de tfidf est',TFrabat['tfidf'][R],'Terme freq',TFrabat['tf'][R])
print()
C,= ComPlusDemande(TFcasa)


print('la competance la plus demande', TFcasa['words'][C],'dans la ville casa la valeur de tfidf est',TFcasa['tfidf'][C],'Terme freq',TFcasa['tf'][C])

la competance la plus demander  c# dans la ville est Rabat la valeur de tfidf est 12.794692470202726 Terme freq 4.0

la competance la plus demande java dans la ville casa la valeur de tfidf est 20.04420749592589 Terme freq 16.0


### Nombre des competences demander selon les ville

In [18]:
def fu(data):
    A=[]
    A1=[]
    for ai in a:
        ii=0
        for i in range(1,len(data['competences_souhaites'])):
            if ai == data['ville'][i]:
                ii =ii+1
            A1 =[ai,ii]
        A=A1+ A
    return A         
A=fu(data1)
A[0]

'Rabat'